<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [58]:
import pandas as pd
import numpy as np


In [59]:
df = pd.read_csv('player_and_values.csv')
df.head()
df['singular_name_short'].unique()

array(['DEF', 'MID', 'FWD', 'GKP'], dtype=object)

In [60]:
gk = df[df['singular_name_short']=='GKP']
defense = df[df['singular_name_short']=='DEF']
mid = df[df['singular_name_short']=='MID']
fwd = df[df['singular_name_short']=='FWD']


df_sorted = gk.append(defense).append(mid).append(fwd)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted.head()

,id_player,web_name,singular_name_short,first_name,second_name,value,mean_points,std_points,count_points,z_score,confidence,per_dollar
0,80,Raya,GKP,David,Raya Martin,4.6,4.285714,2.360387,7,0.932624,4.803845,0.931677
1,353,Lloris,GKP,Hugo,Lloris,5.6,5.000000,4.082483,7,1.391871,3.240370,0.892857
2,559,Ramsdale,GKP,Aaron,Ramsdale,4.5,4.000000,3.633180,6,0.748925,2.696799,0.888889
3,475,Sá,GKP,José,Malheiro de Sá,5.0,4.285714,4.644505,7,0.932624,2.441365,0.857143
4,231,Alisson,GKP,Alisson,Ramses Becker,6.0,5.000000,3.511885,7,1.391871,3.766854,0.833333


In [61]:
costs = df_sorted['value']
val   = df_sorted['mean_points']
std = df_sorted['std_points']
# per_dollar|mean_points

As a first example, consider the solution of the 0/1 knapsack problem: given a set 𝐼 of items, each one
with a weight 𝑤𝑖 and estimated profit 𝑝𝑖
, one wants to select a subset with maximum profit such that the
summation of the weights of the selected items is less or equal to the knapsack capacity 𝑐. Considering a
set of decision binary variables 𝑥𝑖 that receive value 1 if the 𝑖-th item is selected, or 0 if not, the resulting
mathematical programming formulation is:


In [68]:
df_sorted[df_sorted['web_name']=='Jansson']

,id_player,web_name,singular_name_short,first_name,second_name,value,mean_points,std_points,count_points,z_score,confidence,per_dollar
68,76,Jansson,DEF,Pontus,Jansson,4.6,5.428571,2.225395,7,2.159651,6.453979,1.180124


In [75]:
from mip import Model, xsum, maximize, BINARY

# wi = weights
# pi = profit
# c = capacity : total cost (100)


p = list(val.values)
w = list(costs.values)
std_penalty = list(std.values)
c= 82.5
I = range(len(w))
I_g = range(len(gk))
I_d= range(len(defense))
I_m= range(len(mid))
I_f= range(len(fwd))

m = Model("knapsack")
# define vector of chosen players
x = [m.add_var(var_type=BINARY) for i in I]

# objective function
m.objective = maximize(xsum((p[i] * x[i])-np.log(std_penalty[i]) for i in I) )

# cost constraint
m += xsum(w[i] * x[i] for i in I) <= c

#constraint for number of GK
m += xsum(x[i] for i in I_g )==1
#constraint for number of DEF
m += xsum(x[i + len(gk)] for i in I_d )==3
#constraint for number of MID
m += xsum(x[i+ len(gk)+ len(defense)] for i in I_m )==4
#constraint for number of FWD
m += xsum(x[i+ len(gk)+ len(defense)+ len(mid)] for i in I_f )==3

# exclude players by id
# example lloris = 1
ids_to_exclude = [1,279,4,75]
ids_to_exclude = [91,284,68,
                 276,277,279,281]
m += xsum(x[i] for i in ids_to_exclude) == 0


m.optimize()

selected = [i for i in I if x[i].x >= 0.99]
print("selected items: {} ".format(selected))


# for i in selected:
#     print
df_sorted.loc[selected].sort_values('singular_name_short')

C:\Users\212778050\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


selected items: [1, 69, 72, 75, 278, 282, 283, 302, 528, 529, 534] 


,id_player,web_name,singular_name_short,first_name,second_name,value,mean_points,std_points,count_points,z_score,confidence,per_dollar
69,91,Pinnock,DEF,Ethan,Pinnock,4.6,5.428571,3.552330,7,2.159651,4.043163,1.180124
72,256,Cancelo,DEF,João Pedro Cavaco,Cancelo,6.1,6.285714,4.535574,7,2.707839,3.666667,1.030445
75,122,Alonso,DEF,Marcos,Alonso,6.0,5.571429,5.380742,7,2.251016,2.739513,0.928571
528,583,Hwang,FWD,Hee-Chan,Hwang,5.5,6.000000,5.597619,4,2.061870,2.143769,1.090909
529,413,Antonio,FWD,Michail,Antonio,8.0,7.000000,6.733003,7,2.598946,2.750668,0.875000
534,205,Vardy,FWD,Jamie,Vardy,10.4,6.857143,4.017817,7,2.522221,4.515460,0.659341
1,353,Lloris,GKP,Hugo,Lloris,5.6,5.000000,4.082483,7,1.391871,3.240370,0.892857
278,419,Benrahma,MID,Saïd,Benrahma,6.6,6.285714,4.785892,7,2.793099,3.474888,0.952381
282,144,Gallagher,MID,Conor,Gallagher,5.7,4.857143,5.113009,7,1.882926,2.513352,0.852130
283,233,Salah,MID,Mohamed,Salah,12.6,10.000000,4.546061,7,5.159551,5.819877,0.793651


In [79]:
x[2].x

0.0